# Registration
This script will creat "reg" directory in ses-01 directory and get registration infromation from functional space to anatomical space and from anatomical space to standard space. At the end, it will concatinate the results to have transformation from funrtional space to standard space.

Linear registration:      functional space >>> anatomical space (epi_reg from FSL)

Non-Lineare registration: anatomical space >>> standard space (antsRegistration from ANT's)


Converting ANTs output into the FSL format:

c3d_affine_too: https://gist.github.com/MSchnei/240bc49f504ef93540d33b02f12bbe8d

Workbench Command: https://www.humanconnectome.org/software/workbench-command



-----------------------------------------------------------
Script written by Sepideh Tabrik& Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2024.11)

-----------------------------------------------------------

## Anatomical >> Standard space registration

In [ ]:
import glob
import os
from pathlib import Path

# Define data paths
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")
reference_path = Path("/home/mehdi/fsl/data/standard")

# Find subject directories
sub_dirs = list(data_path.glob("sub*/ses-01"))

# Process each subject directory
for curr_sub in sub_dirs:
    subject_id = curr_sub.parts[-2]
    session_id = curr_sub.parts[-1]

    print("1) Creating 'reg' directory.")
    print(f"\t - Current subject is >> {subject_id}/{session_id}")

    reg_directory = curr_sub / "reg"

    # Check if the directory exists and remove if necessary
    if reg_directory.exists():
        print(f"\t {reg_directory} already exists! Removing...")
        os.system(f"rm -r {reg_directory}")

    # Create new reg directory
    print(f"\t Creating directory {reg_directory}...")
    reg_directory.mkdir(parents=True, exist_ok=True)

    # Copy atlas images to reg directory
    print("2) Copying standard images to the reg directory.")
    os.system(f"fslmaths {reference_path}/MNI152_T1_2mm_brain {reg_directory}/standard")
    os.system(f"fslmaths {reference_path}/MNI152_T1_2mm {reg_directory}/standard_head")
    os.system(f"fslmaths {reference_path}/MNI152_T1_2mm_brain_mask_dil {reg_directory}/standard_mask")

    # Copy anatomical images to reg directory
    print("3) Copying anatomical images to reg directory.")
    anat_dir = curr_sub / "anat"

    # Find T1w image
    t1w_files = list(anat_dir.glob("*T1w.nii.gz"))
    if not t1w_files:
        print("Warning: No T1w file found, skipping brain masking step.")
    else:
        t1w_image = t1w_files[0]  # Get first matching file
        print(f"Processing T1w image: {t1w_image}")

        # Copy high-resolution anatomical head
        os.system(f"fslmaths {t1w_image} {reg_directory}/highres_head")

    # Copy mask and brain images
    mask_file = anat_dir / "mask.nii.gz"
    brain_file = anat_dir / "brain.nii.gz"

    if mask_file.exists():
        print(f"Copying anatomical mask: {mask_file} to reg directory...")
        os.system(f"fslmaths {mask_file} {reg_directory}/highres_mask")

        # Apply mask only if T1w exists
        if t1w_files:
            print(f"Applying mask: Multiplying {t1w_image} with {mask_file} to generate highres...")
            os.system(f"fslmaths {t1w_image} -mul {mask_file} {reg_directory}/highres")


    # Change directory to reg
    print("\t Changing the working directory to reg directory.")
    os.chdir(reg_directory)

    # Nonlinear registration using ANTs
    print("4) Running nonlinear registration using ANTs.")
    os.system("antsRegistration --dimensionality 3 --float 0 "
              "--output [highres2standard_,highres2standard.nii.gz] "
              "--interpolation Linear "
              "--winsorize-image-intensities [0.005,0.995] "
              "--use-histogram-matching 0 "
              "--initial-moving-transform [standard.nii.gz,highres.nii.gz,1] "
              "--transform Rigid[0.1] "
              "--metric MI[standard.nii.gz,highres.nii.gz,1,32,Regular,0.25] "
              "--convergence [1000x500x250x100,1e-6,10] "
              "--shrink-factors 8x4x2x1 "
              "--smoothing-sigmas 3x2x1x0vox "
              "--transform Affine[0.1] "
              "--metric MI[standard.nii.gz,highres.nii.gz,1,32,Regular,0.25] "
              "--convergence [1000x500x250x100,1e-6,10] "
              "--shrink-factors 8x4x2x1 "
              "--smoothing-sigmas 3x2x1x0vox "
              "--transform SyN[0.1,3,0] "
              "--metric CC[standard.nii.gz,highres.nii.gz,1,4] "
              "--convergence [100x70x50x20,1e-6,10] "
              "--shrink-factors 8x4x2x1 "
              "--smoothing-sigmas 3x2x1x0vox ")

    # Generate slices and visualization
    os.system("slicer highres2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png "
              "-y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png "
              "-z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; "
              "pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png "
              "+ sli.png + slj.png + slk.png + sll.png highres2standard1.png ; "
              "slicer standard highres2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png "
              "-y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png "
              "-z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; "
              "pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png "
              "+ sli.png + slj.png + slk.png + sll.png highres2standard2.png ; "
              "pngappend highres2standard1.png - highres2standard2.png highres2standard.png; "
              "rm -f sl?.png highres2standard2.png")

    os.system("rm highres2standard1.png")
    print("\n")


### Plot Anatomical >> Atlas registration results

In [ ]:
%matplotlib inline

from IPython.display import display, Image
import matplotlib.pyplot as plt
from pathlib import Path
import glob

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find registration directories
reg_dirs = list(data_path.glob("sub*/ses-01/reg"))

# Process each registration directory
for curr_reg in reg_dirs:
    subject_id = curr_reg.parts[-3]  # Extract subject ID dynamically
    print(f"Processing subject: {subject_id}")

    # Ensure the image file exists before displaying
    image_path = curr_reg / "highres2standard.png"
    if image_path.exists():
        display(Image(filename=str(image_path)))
    else:
        print(f"Warning: {image_path} not found, skipping.")


# copy reg directory for each resting-state sessions

In [ ]:
import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find subject directories
sub_dirs = list(data_path.glob("sub*/ses-0*"))

# Process each subject directory
for curr_sub in sub_dirs:
    subject_id = curr_sub.parts[-2]  # Extract subject ID dynamically
    print(f"Processing subject: {subject_id}")

    # Copy reg directory to different sessions
    reg_src = curr_sub / "reg"

    sessions = curr_sub.glob("func/rest*")
    for curr_sess in sessions:
        exp_name = curr_sess.parts[-1]
        print(f"\t Creating reg directory at {exp_name}.")

        reg_dst = curr_sess / "reg"  # Fixed indentation issue
        reg_dst.mkdir(parents=True, exist_ok=True)  # Ensures the directory is created safely
        
        print(f"\t Copying the reg folder to {exp_name}.")
        if reg_src.exists():
            os.system(f"cp -R {reg_src}/* {reg_dst}")
        else:
            print(f"\t Warning: Source directory {reg_src} does not exist, skipping copy.")

        # Create example_func file using mean function images
        print(f"\t Copying the mean_function as example_func file for {exp_name}.")
        mean_Img = curr_sess / "mean_func.nii.gz"
        mean_Img_Brain = curr_sess / "mean_func_brain.nii.gz"
        mean_Img_Brain_mask = curr_sess / "mean_func_brain_mask.nii.gz"

        for img_file, output_file in [(mean_Img, "example_func.nii.gz"),
                                      (mean_Img_Brain, "example_func_brain.nii.gz"),
                                      (mean_Img_Brain_mask, "example_func_brain_mask.nii.gz")]:
            if img_file.exists():
                os.system(f"cp -R {img_file} {reg_dst}/{output_file}")
            else:
                print(f"\t Warning: {img_file} not found, skipping copy.")

        print()


## Linear registration of Functional data into the Anatomical space 

You must check the results to make sure the registration is done in correct way.


In [ ]:
import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find registration directories
reg_dirs = list(data_path.glob("sub*/ses*/func/rest*/reg"))

# Process each registration directory
for curr_reg in reg_dirs:
    subject_id = curr_reg.parts[-5]
    session_id = curr_reg.parts[-4]
    task_id = curr_reg.parts[-2]

    print(f"Current subject: {subject_id}")
    print(f"\tCurrent session: {session_id}")
    print(f"\tCurrent task: {task_id}")

    # Change path only if it exists
    if curr_reg.exists():
        os.chdir(curr_reg)
    else:
        print(f"\tWarning: {curr_reg} does not exist, skipping.")
        continue

    # FSL Registration: func2highres
    print("\tRunning FSL registration...")

    # Check if epi_reg is applicable
    epi_reg_command = "epi_reg --epi=example_func --t1=highres_head --t1brain=highres --out=example_func2highres"
    flirt_command = "flirt -in example_func_brain -ref highres -out example_func2highres -omat example_func2highres.mat -bins 256 -cost corratio -searchrx 0 0 -searchry 0 0 -searchrz 0 0 -dof 7 -interp trilinear"

    if os.system(epi_reg_command) != 0:
        print("\tWarning: epi_reg failed, falling back to flirt.")
        os.system(flirt_command)

    # Generate transformation matrix
    os.system("convert_xfm -inverse -omat highres2example_func.mat example_func2highres.mat")

    # Generate visualization
    print("\tGenerating visualization...")
    os.system("slicer example_func2highres highres -s 2 "
              "-x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png "
              "-y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png "
              "-z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; "
              "pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png "
              "+ sli.png + slj.png + slk.png + sll.png example_func2highres1.png ; "
              "slicer highres example_func2highres -s 2 "
              "-x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png "
              "-y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png "
              "-z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; "
              "pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png "
              "+ sli.png + slj.png + slk.png + sll.png example_func2highres2.png ; "
              "pngappend example_func2highres1.png - example_func2highres2.png example_func2highres.png; "
              "rm -f sl?.png example_func2highres2.png")

    os.system("rm example_func2highres1.png")
    print()


### Plot example_func >> anatomical space registeration results

In [ ]:
%matplotlib inline

from IPython.display import display, Image
import matplotlib.pyplot as plt
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find registration directories
reg_dirs = list(data_path.glob("sub*/ses*/func/rest*/reg"))

# Process first 10 registration directories
for curr_reg in reg_dirs:
    subject_id = curr_reg.parts[-5]
    session_id = curr_reg.parts[-4]
    task_id = curr_reg.parts[-2]

    print(f"Current subject: {subject_id}, Session: {session_id}, Task: {task_id}")

    # Ensure the image file exists before displaying
    image_path = curr_reg / "example_func2highres.png"
    if image_path.exists():
        display(Image(filename=str(image_path)))
    else:
        print(f"Warning: {image_path} not found, skipping.")



# Conver Ants file to FSL formt

In [ ]:
import glob
import os

data_path = '/mnt/d/Data/Human/ObjectCategorization/analysis'
reg_dirs = glob.glob('%s/sub*/ses*/func/*/reg'%(data_path))

for curr_reg in reg_dirs[1:]:
    
    print(curr_reg)

    # change path
    os.chdir(curr_reg)
    
    #ConvertTransformFile dimensions inputTransfromFile.ext outputTransformFile.ext 
    
    os.system('c3d_affine_tool -itk highres2standard_0GenericAffine.mat -o ants_affine_world.txt')

    os.system('wb_command -convert-affine -from-world ants_affine_world.txt -to-flirt ants_affine_flirt.txt standard.nii.gz highres.nii.gz')

    os.system('convert_xfm -inverse ants_affine_flirt.txt -omat ants_affine_inv_flirt.txt')

#apply X and Y flips to warpfields
#first negate all of them, then take the frames I need
    os.system("wb_command -volume-math '-x' ants_warponly_negative.nii.gz -var x  highres2standard_1Warp.nii.gz")
    #os.system('fslmaths highres2standard_1Warp.nii.gz -mul -1 ants_warponly_negative.nii.gz')
    
    os.system('wb_command -volume-merge ants_warponly_world.nii.gz -volume ants_warponly_negative.nii.gz -subvolume 1 -up-to 2 -volume  highres2standard_1Warp.nii.gz -subvolume 3')

    os.system('wb_command -convert-warpfield -from-world ants_warponly_world.nii.gz -to-fnirt ants_warponly_fnirt.nii.gz standard.nii.gz')

#compose
    os.system('convertwarp --ref=standard.nii.gz --premat=ants_affine_inv_flirt.txt --warp1=ants_warponly_fnirt.nii.gz --out=highres2standard_warp.nii.gz')

#resample to sanity check
    os.system('applywarp --ref=standard.nii.gz --in=highres.nii.gz --warp=highres2standard_warp.nii.gz --out=highres2standard.nii.gz')

    os.system('slicer highres2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard1.png ; slicer standard highres2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard2.png ; pngappend highres2standard1.png - highres2standard2.png highres2standard.png; rm -f sl?.png highres2standard2.png')
    os.system('rm highres2standard1.png')
    #os.system('cp -R highres2standard_warp.nii.gz %s'%(os.path.join(curr_run,'reg')))
    #os.system('cp -R highres2standard.nii.gz %s'%(os.path.join(curr_run,'reg')))
    #os.system('cp -R highres2standard.png %s'%(os.path.join(curr_run,'reg')))
    #os.chdir(os.path.join(curr_run,'reg'))

    os.system('convertwarp --ref=standard --premat=example_func2highres.mat --warp1=highres2standard_warp --out=example_func2standard_warp')
    os.system('applywarp --ref=standard --in=example_func_brain --out=example_func2standard --warp=example_func2standard_warp')
    #os.system('convert_xfm -inverse -omat standard2example_func.mat example_func2standard.mat')
    os.system('slicer example_func2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard1.png ; slicer standard example_func2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard2.png ; pngappend example_func2standard1.png - example_func2standard2.png example_func2standard.png; rm -f sl?.png example_func2standard2.png')
   

# plot registration results: functional >> Atlas space

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Human/ObjectCategorization/analysis'

reg_dirs = glob.glob('%s/sub*/ses*/func/*/reg'%(data_path))
for curr_reg in reg_dirs[0:10]:
    
    reg_splits = curr_reg.split('/')
    #print(reg_dir)
    #img = mpimg.imread(os.path.join(reg_dir, 'example_func2standard.png' ))
    print(curr_reg.split('/')[-5], curr_reg.split('/')[-4],  curr_reg.split('/')[-2])
    #imgplot = plt.imshow(img)
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'example_func2standard.png' )))

# Invert the warp from MNI 2 functional space

In [ ]:
import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find registration directories
sub_dirs = list(data_path.glob("sub*/ses-0*/func/rest*/reg"))

# Process each registration directory
for curr_reg in sub_dirs:
    print(f"Processing: {curr_reg}")

    # Change directory only if it exists
    if curr_reg.exists():
        os.chdir(curr_reg)
    else:
        print(f"\tWarning: {curr_reg} does not exist, skipping.")
        continue

    # Define warp file paths
    warp_file = curr_reg / "example_func2standard_warp.nii.gz"
    ref_image = curr_reg / "example_func_brain.nii.gz"
    output_warp = curr_reg / "standard2example_func_warp.nii.gz"
    output_image = curr_reg / "standard2example_func.nii.gz"

    # Ensure warp file exists before running transformations
    if warp_file.exists():
        print("\tRunning inverse warp...")
        os.system(f"invwarp --ref={ref_image} --warp={warp_file} --out={output_warp}")

        print("\tApplying warp transformation...")
        os.system(f"applywarp --ref={ref_image} --in=standard --out={output_image} --warp={output_warp}")
    else:
        print(f"\tWarning: Warp file {warp_file} not found, skipping transformations.")
